In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Loading Necessary Packages

from matplotlib.pyplot import imshow,plot
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(11.7,5.27)})
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization

In [ ]:
#Loading Test/train/validation Data

df_train_raw = pd.read_csv(os.path.join('/kaggle/input/Kannada-MNIST/train.csv'))
df_valid_raw = pd.read_csv(os.path.join('/kaggle/input/Kannada-MNIST/Dig-MNIST.csv'))
df_test_raw = pd.read_csv(os.path.join('/kaggle/input/Kannada-MNIST/test.csv'))

In [ ]:
#Merging Train & Validation Set

df_train_raw=pd.concat([df_train_raw,df_valid_raw])
df_train_raw.shape

In [ ]:
imshow(df_train_raw.iloc[255,1:].values.reshape(28,28),cmap='gray')

In [ ]:
#Splitting Dependent and Independent Variables

df_train_x=df_train_raw.iloc[:,1:]
df_train_y=df_train_raw.iloc[:,0]
print(df_train_x.shape)
print(df_train_y.shape)

In [ ]:
image_x=28
image_y=28

In [ ]:
# Resizing Numpy array to Proper shape 

x=np.array(df_train_x).reshape(df_train_x.shape[0],image_x,image_y,1)
x.shape

In [ ]:
#scaling the Images

x = x.astype('float32')
x/=255.0

In [ ]:
#converting  Target  classes to one hot encoding format

from keras.utils import to_categorical
df_train_y=to_categorical(df_train_y)
df_train_y.shape

In [ ]:
# Using ExponentialDecay functionality which will decrease the learning rate exponentially with epocs 

from keras.optimizers.schedules import ExponentialDecay
initial_learning_rate=0.001
def exp_decay():
   lr_schedule = ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=0.96,
        staircase=True)
   return lr_schedule

In [ ]:
# Function which will help visualize pixels in HUman readable images

def vis(dataset,samples,label_col,legend=False,cmap='gray_r',cbar=False,transpose=True,prediction=False,dict_name=None):
    samples=dataset.iloc[samples].reset_index(drop=True) # Picks n random samples from the dataset
    labels=samples[label_col].values
    samples.drop(label_col,axis=1,inplace=True)
    if prediction==True:
        preds=samples['Prediction'].values
        samples.drop('Prediction',axis=1,inplace=True)
    fig, ax = plt.subplots(5,(len(samples)//5)+1,sharey=True,sharex=True)
    for i in range(len(samples)):
        pixels=samples.iloc[i].values
        pixels=pixels.reshape((28,28))
        if transpose==True:
            pixels=pixels.transpose()
        sns.heatmap(pixels,cmap=cmap,cbar=cbar,ax=ax.flatten()[i])
        ax.flatten()[i].axes.get_xaxis().set_visible(False)
        ax.flatten()[i].axes.get_yaxis().set_visible(False)
    plt.tight_layout()
    plt.show()
    if legend==True:
        if dict_name!=None:
            labels=[dict_name[i] for i in labels]
        if prediction==True:
            print('The images represent items with labels {} which were predicted to be {}.'.format(labels,preds))
        else:
            print('The images represent items with labels {}.'.format(labels))

In [ ]:
vis(df_train_raw,list(range(0,10)),label_col='label',legend=True)


In [ ]:
# Using Early stopping functionality for stopping the training where is not much change in validation Loss

from keras.callbacks import EarlyStopping

def early_stopping():
    return EarlyStopping(monitor="val_loss",patience=10,verbose=1)

In [ ]:
# This will help the network reduce the learning rate as the gradint is  reaching minima 

from keras.callbacks import ReduceLROnPlateau

def reduce_lr_platue():
    return ReduceLROnPlateau(
        monitor='val_loss', factor=0.1, patience=5, verbose=1,
    )

In [ ]:
df_train_y.shape

In [ ]:
from sklearn.model_selection import train_test_split
seed=7
np.random.seed(seed)
X_train, X_test, y_train, y_test = train_test_split(x, df_train_y, test_size=0.10, random_state=seed, shuffle=True)

In [ ]:
#Create custom convnet
# after multiple architectures this design was giving the best results

from keras.layers import Input,maximum
from keras import Model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense,BatchNormalization,AveragePooling2D
from keras.optimizers import Adam,RMSprop

inp_layer = Input(shape=(image_x, image_y, 1))
conv_1=Conv2D(64, kernel_size=(1, 1), padding='same', activation='relu')(inp_layer)
max_1=MaxPooling2D(2,2)(conv_1)

conv_2=Conv2D(64, kernel_size=(2, 2), padding='same', activation='relu')(inp_layer)
max_2=MaxPooling2D(2,2)(conv_2)

conv_3=Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu')(inp_layer)
max_3=MaxPooling2D(2,2)(conv_3)

merged = maximum([max_1, max_2, max_3])
bn=BatchNormalization()(merged)
dr=Dropout(0.20)(bn)

conv_1=Conv2D(128, kernel_size=(1, 1), padding='same', activation='relu')(dr)
max_1=MaxPooling2D(2,2)(conv_1)

conv_2=Conv2D(128, kernel_size=(2, 2), padding='same', activation='relu')(dr)
max_2=MaxPooling2D(2,2)(conv_2)

conv_3=Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu')(dr)
max_3=MaxPooling2D(2,2)(conv_3)

merged = maximum([max_1, max_2, max_3])
bn=BatchNormalization()(merged)
dr=Dropout(0.20)(bn)

model= Model(inp_layer, dr)

# add created model grapg in sequential model

model_cnn2 = Sequential()
model_cnn2.add(model)

model_cnn2.add(Conv2D(256, kernel_size=3, activation='relu'))
model_cnn2.add(BatchNormalization())
model_cnn2.add(Flatten())

model_cnn2.add(Dropout(0.25))
model_cnn2.add(Dense(512,activation='relu'))
model_cnn2.add(BatchNormalization())
model_cnn2.add(Dropout(0.20))
model_cnn2.add(Dense(64,activation='relu'))
model_cnn2.add(BatchNormalization())
model_cnn2.add(Dropout(0.10))
model_cnn2.add(Dense(10,activation='softmax'))
model_cnn2.summary()
optimizer = RMSprop(lr=0.001)
model_cnn2.compile(loss='categorical_crossentropy',metrics="accuracy"
                , optimizer=optimizer)

In [ ]:
'''from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense,BatchNormalization,AveragePooling2D
from keras.optimizers import Adam

model_cnn1= Sequential()
model_cnn1.add(Conv2D(64, kernel_size=(5, 5),activation='relu',input_shape=(image_x,image_y,1)))
model_cnn1.add(Conv2D(64, kernel_size=(3, 3),padding='same',activation='relu'))
model_cnn1.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn1.add(Dropout(0.20))
model_cnn1.add(Conv2D(128, kernel_size=(5, 5),activation='relu'))
model_cnn1.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model_cnn1.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn1.add(Flatten())
model_cnn1.add(Dense(128,activation='relu'))
model_cnn1.add(BatchNormalization())
model_cnn1.add(Dropout(0.25))
model_cnn1.add(Dense(32,activation='relu'))
model_cnn1.add(BatchNormalization())
model_cnn1.add(Dropout(0.10))
model_cnn1.add(Dense(10,activation='softmax'))
model_cnn1.summary()
model_cnn1.compile(loss='categorical_crossentropy',metrics="accuracy"
                , optimizer='rmsprop')'''

In [ ]:
#Model Training
model_output_5 = model_cnn2.fit(X_train,y_train,epochs=40,batch_size=128,validation_data=(X_test,y_test),callbacks=[reduce_lr_platue(),early_stopping()])

In [ ]:
# Plotting Loss and accuracy graph

fig, ax = plt.subplots(1,2)
ax[0].plot(list(range(len(model_output_5.history["loss"]))), model_output_5.history["loss"], 'r--')
ax[0].plot(list(range(len(model_output_5.history["loss"]))),model_output_5.history["val_loss"], 'b-')
ax[0].legend(['test_Loss', 'val_loss'])
ax[0].set_title('Loss')
ax[1].plot(list(range(len(model_output_5.history["loss"]))), model_output_5.history["accuracy"], 'r--')
ax[1].plot(list(range(len(model_output_5.history["loss"]))),model_output_5.history["val_accuracy"], 'b-')
ax[1].legend(['test_accuracy', 'val_accuracy'])
ax[1].set_title('Accuracy')
plt.show()

In [ ]:
# Validating in validation data set
x_pred_total=model_cnn2.predict(X_test)

In [ ]:
#Analysing validation set Predictions

y_list=[]
train_results=pd.DataFrame()
for i in range(len(x_pred_total)):
     y_list.append(np.argmax(x_pred_total[i]))
train_results["ypred"] = y_list
y_list=[]
for i in range(len(y_test)):
     y_list.append(np.argmax(y_test[i]))
train_results["yactual"] = y_list


In [ ]:
n_match = train_results[train_results["ypred"]!=train_results["yactual"]]
print(n_match.shape)
n_match.groupby(by="yactual").count()

In [ ]:
#Comparing the Prediction with image
print(df_train_raw.iloc[14097,0])
imshow(df_train_raw.iloc[14097,1:].values.reshape(28,28),cmap='gray')

In [ ]:
# converting test set to proper format for prediction

df_test=df_test_raw.iloc[:,1:] 
x = df_test.astype('float32') 
test=np.array(x).reshape(x.shape[0],image_x,image_y,1) 
test/=255.0

In [ ]:
#Predicting on test_set
y_pred=model_cnn2.predict(test)

In [ ]:
# Saving the results for submission

y_list=[]
for i in range(len(y_pred)):
     y_list.append(np.argmax(y_pred[i]))
result=df_test_raw[["id"]]
result["label"] = y_list
result.to_csv("submission.csv",index=False)


In [ ]:
result